# Rekognition via VPC

In [1]:
import time
import boto3
import glob

client_rek = boto3.client('rekognition')
client_s3 = boto3.client('s3')

In [2]:
def get_all_images(bucket_name, prefix):
    image_list = []
    response = response = client_s3.list_objects_v2(
        Bucket= bucket_name,
        Prefix= prefix)
    for s3_object in response["Contents"]:
        if (s3_object["Size"] > 0):
            image_list.append(s3_object["Key"])
    return image_list

In [3]:
bucket_name = "rkg-latency-tests"

## Detect Labels

### Local Images

In [4]:
def process_local_images_labels(folder):
    images = glob.glob('{}/*'.format(folder))
    start = time.time()
    for image in images:
        with open(image, 'rb') as imageBytes:
            response = client_rek.detect_labels(
                Image={'Bytes': imageBytes.read()}
            )
    total_time = time.time() - start
    return total_time   

In [5]:
total_time = process_local_images_labels("media/labels")
print("Time to process all images: {}".format(total_time))

Time to process all images: 1.2122044563293457


### Images in S3

In [6]:
def process_s3_images_labels(bucket_name):
    image_list = get_all_images(bucket_name, "labels")
    start = time.time()
    for image in image_list:
        response = client_rek.detect_labels(
            Image={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': image
                }
            }
        )
    total_time = time.time() - start
    return total_time 

In [7]:
total_time = process_s3_images_labels(bucket_name)
print("Time to process all images: {}".format(total_time))  

Time to process all images: 1.5267958641052246


## Detect Content Moderation

### Local Images

In [8]:
def process_local_images_moderation(folder):
    images = glob.glob('{}/*'.format(folder))
    start = time.time()
    for image in images:
        with open(image, 'rb') as imageBytes:
            response = client_rek.detect_moderation_labels(
                Image={'Bytes': imageBytes.read()}
            )
    total_time = time.time() - start
    return total_time   

In [9]:
total_time = process_local_images_moderation("media/moderation")
print("Time to process all images: {}".format(total_time))

Time to process all images: 1.7448678016662598


### Images in S3

In [10]:
def process_s3_images_moderation(bucket_name):
    image_list = get_all_images(bucket_name,"moderation")
    start = time.time()
    for image in image_list:
        response = client_rek.detect_moderation_labels(
            Image={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': image
                }
            }
        )
    total_time = time.time() - start
    return total_time

In [11]:
total_time = process_s3_images_moderation(bucket_name)
print("Time to process all images: {}".format(total_time))  

Time to process all images: 1.8632268905639648


## Detect Faces

### Local Images

In [12]:
def process_local_images_faces(folder):
    images = glob.glob('{}/*'.format(folder))
    start = time.time()
    for image in images:
        with open(image, 'rb') as imageBytes:
            response = client_rek.detect_labels(
                Image={'Bytes': imageBytes.read()}
            )
    total_time = time.time() - start
    return total_time

In [13]:
total_time = process_local_images_faces("media/faces")
print("Time to process all images: {}".format(total_time))       

Time to process all images: 1.6686251163482666


### Images in S3

In [14]:
def process_s3_images_faces(bucket_name):
    image_list = get_all_images(bucket_name, "faces")
    start = time.time()
    for image in image_list:
        response = client_rek.detect_labels(
            Image={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': image
                }
            }
        )
    total_time = time.time() - start
    return total_time

In [15]:
total_time = process_s3_images_faces(bucket_name)
print("Time to process all images: {}".format(total_time)) 

Time to process all images: 1.919269323348999


## Average

In [17]:
total_labels_local = 0
total_labels_s3 = 0
total_moderation_local = 0
total_moderation_s3 = 0
total_faces_local = 0
total_faces_s3 = 0

for x in range(20):
    total_labels_local += round(process_local_images_labels("media/labels"),3)
    total_labels_s3 += round(process_s3_images_labels(bucket_name),3)
    total_moderation_local += round(process_local_images_moderation("media/moderation"),3)
    total_moderation_s3 += round(process_s3_images_labels(bucket_name),3)
    total_faces_local += round(process_s3_images_labels(bucket_name),3)
    total_faces_s3 += round(process_local_images_faces("media/faces"),3)
    
print("Average of 20 calls to labels using local images via VPC is {}".format(total_labels_local/20))
print("Average of 20 calls to labels using S3 via VPC is {}".format(total_labels_s3/20))
print("Average of 20 calls to moderation using local images via VPC is {}".format(total_moderation_local/20))
print("Average of 20 calls to moderation using S3 via VPC is {}".format(total_moderation_s3/20))
print("Average of 20 calls to faces using local images via VPC is {}".format(total_faces_local/20))
print("Average of 20 calls to faces using S3 via VPC is {}".format(total_faces_s3/20))


Average of 20 calls to labels using local images via VPC is 1.32235
Average of 20 calls to labels using S3 via VPC is 1.2789
Average of 20 calls to moderation using local images via VPC is 1.4919000000000002
Average of 20 calls to moderation using S3 via VPC is 1.32515
Average of 20 calls to faces using local images via VPC is 1.2424
Average of 20 calls to faces using S3 via VPC is 1.32985
